In [3]:
import numpy as np
import pandas as pd
import csv
import sys
import os

from sys import platform as sys_pf
if sys_pf == 'darwin':
    import matplotlib
    matplotlib.use("TkAgg")
    import matplotlib.pyplot as plt
    
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import warnings
import pickle

max_iters = 513
n_estimators = 10

def logreg(x,y,filename):

   # Model output file name
   file = (os.path.splitext(filename))[0]
   fname = './models/lr_' + file +'/'

   # File for writing precision,recall, f-measure scores for fraud transactions
   f = open('./prf/lr_'+ file + '_prf' +'.txt' ,'w')
   f.write('precision,recall,f-score \n')

   # Stratified sampling based on Y
   X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y , test_size=0.30, random_state=42)

   # Create 15% validation set and 15% test set split
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,stratify=y_test , test_size=0.50, random_state=42)
   
   #Iterations
   it = 1
   
   # Run training algorithm for multiple class weights
   while it < max_iters:
       cw = {}
       cw[0] = 1
       cw[1] = it
       # Train
       print('**************************************')
       print("Iteration number  " , it)
       lr = LogisticRegression(class_weight = cw)
       print('Class weights ', cw)
       lr.fit(X_train,y_train)

       # Save trained model to disk
       name = fname + str(cw[1]) + '.sav'
       pickle.dump(lr, open(name, 'wb'))

       # Predict on validation data
       y_val_pred = lr.predict(X_val)
       print('Performance on validation data - Confusion matrix')
       print(confusion_matrix(y_val,y_val_pred))
   
       precision,recall,fscore,support=score(y_val,y_val_pred,average=None)
       print('Precision, Recall, F-score, Support  on validation data' )
       print("Precision" , precision)
       print("Recall" , recall)
       print("F-score" , fscore)
       print("Support" , support)

       p1 = precision[1]
       r1 = recall[1]
       f1 = fscore[1]

       f.write(str(p1) +','+ str(r1) + ',' + str(f1) + '\n') 
       it += 1

   f.close()

def run():
   filename = 'full_data.csv'
   df = pd.read_csv(filename, usecols = [8,9,10,11,12,13,14] , header = 0)
   
   results = list(map(int, df['Anomaly'])) 
   print('Number of fraudulent transactions ' , sum(results))

   features = ['Transaction_Type', 'Airfare', 'Lodging', 'Meals', 'Other_Transportation', 'Other_Expenses']
   targets = ['Anomaly']

   # Separating out the features and target variables
   X = df.loc[:, features].values
   y = df.loc[:, targets].values

    
   from sklearn.preprocessing import LabelEncoder, OneHotEncoder
   labelencoder_X = LabelEncoder()
   X[:,0] = labelencoder_X.fit_transform(X[:,0])
   onehotencoder = OneHotEncoder(categorical_features = [0])
   x = onehotencoder.fit_transform(X).toarray() 

   scaler = preprocessing.StandardScaler()
   x = scaler.fit_transform(x) 
    
   y  = [i for j in y for i in j]
   
   #Ignore warnings
   warnings.filterwarnings("ignore", category=FutureWarning)

   print("***********Logistic Regression**********")
   logreg(x,y,filename)
  
run()

Number of fraudulent transactions  8213


/home/sandisk/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


***********Logistic Regression**********
**************************************
Iteration number   1
Class weights  {0: 1, 1: 1}
Performance on validation data - Confusion matrix
[[953110     51]
 [   741    491]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99922315 0.90590406]
Recall [0.99994649 0.39853896]
F-score [0.99958469 0.5535513 ]
Support [953161   1232]
**************************************
Iteration number   2
Class weights  {0: 1, 1: 2}
Performance on validation data - Confusion matrix
[[953044    117]
 [   662    570]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99930587 0.82969432]
Recall [0.99987725 0.46266234]
F-score [0.99959148 0.59405941]
Support [953161   1232]
**************************************
Iteration number   3
Class weights  {0: 1, 1: 3}
Performance on validation data - Confusion matrix
[[952973    188]
 [   628    604]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99934144 0.762626

Performance on validation data - Confusion matrix
[[951474   1687]
 [   418    814]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99956087 0.32546981]
Recall [0.9982301  0.66071429]
F-score [0.99889504 0.43611037]
Support [953161   1232]
**************************************
Iteration number   25
Class weights  {0: 1, 1: 25}
Performance on validation data - Confusion matrix
[[951381   1780]
 [   413    819]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99956608 0.3151212 ]
Recall [0.99813253 0.66477273]
F-score [0.99884879 0.4275646 ]
Support [953161   1232]
**************************************
Iteration number   26
Class weights  {0: 1, 1: 26}
Performance on validation data - Confusion matrix
[[951288   1873]
 [   407    825]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99957234 0.30578206]
Recall [0.99803496 0.66964286]
F-score [0.99880306 0.41984733]
Support [953161   1232]
**********************************

Performance on validation data - Confusion matrix
[[949411   3750]
 [   352    880]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99962938 0.19006479]
Recall [0.99606572 0.71428571]
F-score [0.99784437 0.30023883]
Support [953161   1232]
**************************************
Iteration number   49
Class weights  {0: 1, 1: 49}
Performance on validation data - Confusion matrix
[[949316   3845]
 [   351    881]]
Precision, Recall, F-score, Support  on validation data
Precision [0.9996304  0.18641557]
Recall [0.99596605 0.7150974 ]
F-score [0.99779486 0.29573682]
Support [953161   1232]
**************************************
Iteration number   50
Class weights  {0: 1, 1: 50}
Performance on validation data - Confusion matrix
[[949219   3942]
 [   349    883]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99963246 0.18300518]
Recall [0.99586429 0.71672078]
F-score [0.99774482 0.29156348]
Support [953161   1232]
**********************************

...

Precision, Recall, F-score, Support  on validation data
Precision [0.99985585 0.0346229 ]
Recall [0.96785118 0.89204545]
F-score [0.98359324 0.06665858]
Support [953161   1232]
**************************************
Iteration number   402
Class weights  {0: 1, 1: 402}
Performance on validation data - Confusion matrix
[[922449  30712]
 [   133   1099]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99985584 0.0345478 ]
Recall [0.96777879 0.89204545]
F-score [0.98355585 0.06651938]
Support [953161   1232]
**************************************
Iteration number   403
Class weights  {0: 1, 1: 403}
Performance on validation data - Confusion matrix
[[922388  30773]
 [   133   1099]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99985583 0.03448168]
Recall [0.96771479 0.89204545]
F-score [0.98352279 0.06639681]
Support [953161   1232]
**************************************
Iteration number   404
Class weights  {0: 1, 1: 404}
Performance on validati

Performance on validation data - Confusion matrix
[[921038  32123]
 [   126   1106]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99986322 0.03328418]
Recall [0.96629845 0.89772727]
F-score [0.98279434 0.0641885 ]
Support [953161   1232]
**************************************
Iteration number   426
Class weights  {0: 1, 1: 426}
Performance on validation data - Confusion matrix
[[920992  32169]
 [   126   1106]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99986321 0.03323817]
Recall [0.96625019 0.89772727]
F-score [0.98276937 0.06410294]
Support [953161   1232]
**************************************
Iteration number   427
Class weights  {0: 1, 1: 427}
Performance on validation data - Confusion matrix
[[920929  32232]
 [   126   1106]]
Precision, Recall, F-score, Support  on validation data
Precision [0.9998632  0.03317536]
Recall [0.9661841  0.89772727]
F-score [0.98273518 0.06398612]
Support [953161   1232]
******************************

Precision, Recall, F-score, Support  on validation data
Precision [0.99986954 0.03214244]
Recall [0.96487057 0.9025974 ]
F-score [0.98205833 0.06207436]
Support [953161   1232]
**************************************
Iteration number   449
Class weights  {0: 1, 1: 449}
Performance on validation data - Confusion matrix
[[919631  33530]
 [   120   1112]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99986953 0.03209976]
Recall [0.96482231 0.9025974 ]
F-score [0.98203333 0.06199476]
Support [953161   1232]
**************************************
Iteration number   450
Class weights  {0: 1, 1: 450}
Performance on validation data - Confusion matrix
[[919566  33595]
 [   120   1112]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99986952 0.03203965]
Recall [0.96475412 0.9025974 ]
F-score [0.981998   0.06188263]
Support [953161   1232]
**************************************
Iteration number   451
Class weights  {0: 1, 1: 451}
Performance on validati

Performance on validation data - Confusion matrix
[[918271  34890]
 [   117   1115]]
Precision, Recall, F-score, Support  on validation data
Precision [0.9998726  0.03096792]
Recall [0.96339548 0.90503247]
F-score [0.98129517 0.05988667]
Support [953161   1232]
**************************************
Iteration number   473
Class weights  {0: 1, 1: 473}
Performance on validation data - Confusion matrix
[[918212  34949]
 [   117   1115]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99987259 0.03091726]
Recall [0.96333358 0.90503247]
F-score [0.98126306 0.05979193]
Support [953161   1232]
**************************************
Iteration number   474
Class weights  {0: 1, 1: 474}
Performance on validation data - Confusion matrix
[[918136  35025]
 [   117   1115]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99987258 0.03085224]
Recall [0.96325385 0.90503247]
F-score [0.98122169 0.05967034]
Support [953161   1232]
******************************

Precision, Recall, F-score, Support  on validation data
Precision [0.99988223 0.03008887]
Recall [0.96198753 0.91233766]
F-score [0.9805689  0.05825645]
Support [953161   1232]
**************************************
Iteration number   496
Class weights  {0: 1, 1: 496}
Performance on validation data - Confusion matrix
[[916883  36278]
 [   107   1125]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99988331 0.0300778 ]
Recall [0.96193927 0.91314935]
F-score [0.98054435 0.05823735]
Support [953161   1232]
**************************************
Iteration number   497
Class weights  {0: 1, 1: 497}
Performance on validation data - Confusion matrix
[[916824  36337]
 [   106   1126]]
Precision, Recall, F-score, Support  on validation data
Precision [0.9998844  0.03005632]
Recall [0.96187737 0.91396104]
F-score [0.98051271 0.05819873]
Support [953161   1232]
**************************************
Iteration number   498
Class weights  {0: 1, 1: 498}
Performance on validati